In [1]:
import os, sys
sys.path.append(os.path.abspath(os.environ['KALE']))
from kale.jupyter import *
from sklearn import model_selection as ms
import matplotlib.pyplot as plt



%matplotlib inline
%load_ext autoreload
%autoreload 2

%load_ext autotime

quick = Quick(cache = FrameCache('MI'), stim_cache=StimCache())

/Users/tiatummino/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
Lookups.runs(Experiments.name == 'retest :: DOW compounds :: tia :: transfer plate with ideal doses 2')


In [12]:
all_DOW = Lookups.runs(Experiments.name == 'retest :: DOW compounds :: tia :: transfer plate with ideal doses 2')
all_DOW = all_DOW['id'].tolist()
all_DOW.remove(5486)
all_DOW.remove(5499)
len(all_DOW)

8

time: 2.52 s


# Fingerprinting Algorithm (broken down assay by assay)

In [97]:
def fingerprint_run(run_id):
    run = Lookups.runs(run_id)
    battery_name = run['battery'][0]
    
    app_frame = pd.DataFrame(AppFrame.of(battery_name))
    
    
    ct = 0
    
    df = quick.df(run_id)
    
    for index, row in app_frame.iterrows():

        start_ms = row['start_ms']
        

        end_ms = row['end_ms']
        
        name = row['simplified_name']

        frames = df.slice_ms(start_ms = start_ms, end_ms = end_ms ).features()
        frames_grouped = frames.groupby(by = frames.index).mean()


        control = frames_grouped[frames_grouped.index.str.contains('solvent')]
        control_sd = int(np.std(control, axis = 1)) 
        control_mean = int(control.mean(axis = 1))
        

     
        
        if ct == 0: 

            beyond_dict = dict()
            
            for i in range(0,len(frames_grouped.index)):

                non_control = pd.DataFrame((frames_grouped.iloc[i,:])).T

                non_control_mean = int(np.mean(non_control, axis = 1))


                if (non_control_mean > (control_mean +(control_sd))):

                    beyond_dict.update({frames_grouped.index[i]:[run_id,1]})
                elif (non_control_mean <(control_mean -(control_sd))): 
                    beyond_dict.update({frames_grouped.index[i]:[run_id, -1]})
                else:
                    beyond_dict.update({frames_grouped.index[i]:[run_id,0]})
            beyond_df = pd.DataFrame.from_dict(beyond_dict, orient = 'index', columns = ['run_id',name])
            
            ct = ct + 1

        else: 
            beyond_dict = dict()
            
            for i in range(0,len(frames_grouped.index)):

                non_control = pd.DataFrame((frames_grouped.iloc[i,:])).T

                non_control_mean = int(np.mean(non_control, axis = 1))


                if (non_control_mean > (control_mean +(control_sd))):

                    beyond_dict.update({frames_grouped.index[i]:1})
                elif (non_control_mean <(control_mean -(control_sd))): 
                    beyond_dict.update({frames_grouped.index[i]:-1})
                else:
                    beyond_dict.update({frames_grouped.index[i]:0})
            append_df = pd.DataFrame.from_dict(beyond_dict, orient = 'index', columns = [name])
            
            if ct == 1: 
            
            
                fingerprint_table = pd.concat([beyond_df, append_df], 
                                  axis = 1)
                ct = ct + 1
            else: 
                fingerprint_table = pd.concat([fingerprint_table, append_df], 
                              axis = 1)
                ct = ct + 1
                
                
      


    return fingerprint_table


time: 12.3 ms


# Example usage of the fingerprint algorithm 

In [ ]:
fprint = fingerprint_run(5660)
fprint

# Correlate a reference fingerprint to fingerprints from a plate
## Not the most effective becasue it finds patterns, not exact matches

In [67]:
#Doesn't work great becuase not sensitive to magnitude, so it just matches the patterns
def correlate_fingerprint(run_id, reference_id, ideal_dose):
    query = fingerprint_run(run_id)
    query = query[query.index.str.contains(ideal_dose)]
    reference = fingerprint_run(reference_id)
    
    
    
    lt = dict()
    
    for i in range(0,len(reference.index)):
        
        coeff = np.corrcoef(query.iloc[0,:], reference.iloc[i,:])
        coeff = (1-coeff[0][1])
        lt.update({str(i) + ' ' + str(query.index[0]):[reference.index[i],run_id, coeff]} )

    lt = pd.DataFrame.from_dict(lt, orient = 'index', columns = ['reference','run_id','corr_score'])
    lt.sort_values(by=['corr_score'], ascending = True, inplace = True)
    return lt
#e.g. 
#data = correlate_fingerprint(5660,5506,'50.0µM' ) 

time: 9.14 ms


# Find the percent of assays where the fingerprint is the same
## Works best at the moment 

In [96]:
def compare_fingerprints(query_run_id, reference_id, query_ideal_dose):

    query = fingerprint_run(query_run_id)
    query = query[query.index.str.contains(query_ideal_dose)]
    reference = fingerprint_run(reference_id)
    
    
    lt = dict()
    for i in range(0,len(reference.index)):
        

        raw_score = 0
        for n in range(0,len(reference.columns)):
            if (query.iloc[0,n] == reference.iloc[i,n]):
                raw_score = raw_score + 1
            else: 
                continue
        percent_sim = (raw_score/(len(reference.columns)))

        lt.update({str(i) + ' ' + str(query.index[0]):[reference.index[i],query_run_id, 
                                                       reference_id, raw_score, percent_sim]} )


    lt = pd.DataFrame.from_dict(lt, orient = 'index', columns = ['reference', 'query_run_id',
                                                                 'reference_id','raw_score',
                                                                 'percent_similarity'])
    lt.sort_values(by = 'percent_similarity', ascending = False, inplace = True)
    return lt 
    

time: 6.69 ms


In [95]:
compare_fingerprints(5660,5506,'50.0µM' )

,reference,query_run_id,query_reference_id,raw_score,percent_similarity
5 33714 (50.0µM),endosulfan (0.195µM),5660,5506,37,0.770833
2 33714 (50.0µM),buprofezin (6.25µM),5660,5506,25,0.520833
0 33714 (50.0µM),acetamiprid (800.0µM),5660,5506,21,0.437500
3 33714 (50.0µM),chlorantraniliprole (25.0µM),5660,5506,17,0.354167
10 33714 (50.0µM),ivermectin (0.195µM),5660,5506,15,0.312500
12 33714 (50.0µM),pymetrozin (1.56µM),5660,5506,15,0.312500
13 33714 (50.0µM),solvent (-),5660,5506,15,0.312500
14 33714 (50.0µM),thiamethoxam (0.195µM),5660,5506,15,0.312500
7 33714 (50.0µM),flonicamid (0.195µM),5660,5506,14,0.291667
9 33714 (50.0µM),indoxacarb (6.25µM),5660,5506,12,0.250000


time: 17.7 s
